In [2]:
import cv2 as cv
import numpy as np
from scipy.signal import wiener
from scipy import ndimage as nd
from fcmeans import FCM as f
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from skimage import data, img_as_float,io
from skimage.metrics import peak_signal_noise_ratio

import tensorflow.keras.backend as K   

def noise_remFASTNlMeanWeiner(image):
    img=image
    newimg=cv.fastNlMeansDenoisingColored(img,None,3,3,7,21)
    smoothimg=wiener(img,mysize=5)
    smoothimg=np.array(smoothimg,dtype=np.uint8)
    return smoothimg

def noise_remMedianWeiner(image):

    img_noisy1=cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    m, n = img_noisy1.shape

    nimg = np.zeros([m, n])

    for i in range(1, m-1):
        for j in range(1, n-1):
            temp = [img_noisy1[i-1, j-1],
                   img_noisy1[i-1, j],
                   img_noisy1[i-1, j + 1],
                   img_noisy1[i, j-1],
                   img_noisy1[i, j],
                   img_noisy1[i, j + 1],
                   img_noisy1[i + 1, j-1],
                   img_noisy1[i + 1, j],
                   img_noisy1[i + 1, j + 1]]

            temp = sorted(temp)
            nimg[i, j]= temp[4]
    nimg = nimg.astype(np.uint8)
    img=cv.cvtColor(nimg,cv.COLOR_GRAY2BGR) 
    smoothimg=wiener(img,mysize=5)
    smoothimg=np.array(smoothimg,dtype=np.uint8)
    return smoothimg


def contrast_enhanceWMean(image):
    cimg=noise_remFASTNlMeanWeiner(image)
    h,s,v=cv.split(cimg)
    clahe=cv.createCLAHE(clipLimit=5)
    final=clahe.apply(v)
    merged_hsv=cv.merge((h,s,v))
    getImg=cv.cvtColor(merged_hsv,cv.COLOR_HSV2BGR)
    return getImg

def contrast_enhanceWMedian(image):
    cimg=noise_remMedianWeiner(image)
    h,s,v=cv.split(cimg)
    clahe=cv.createCLAHE(clipLimit=5)
    final=clahe.apply(v)
    merged_hsv=cv.merge((h,s,v))
    getImg=cv.cvtColor(merged_hsv,cv.COLOR_HSV2BGR)
    return getImg

def CLAHEcontrastEnhance(img):
    h,s,v=cv.split(img)
    clahe=cv.createCLAHE(clipLimit=5)
    final=clahe.apply(v)
    merged_hsv=cv.merge((h,s,v))
    getImg=cv.cvtColor(merged_hsv,cv.COLOR_HSV2BGR)
    return getImg

def HistContrastEnhance(img):
    h,s,v=cv.split(img)
    dst = cv.equalizeHist(v)
    merged_hsv=cv.merge((h,s,dst))
    enhanced_image = cv.cvtColor(merged_hsv, cv.COLOR_HSV2RGB)

    return enhanced_image
    
    
    
def Otsu_threshold(image):
    img=cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    thresval,threshimg=cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    return (thresval,threshimg)


def kmeans(image,Clusters):
    Z = image.reshape((-1,3))
    Z = np.float32(Z)
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 21, 1.0)
    K=Clusters
    ret,label,center=cv.kmeans(Z,K,None,criteria,10,cv.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((image.shape))
    return res2

def fuzzycMeans(image,Clusters):
    
    Z = image.reshape((-1,3))

    Z = np.float32(Z)

    my_model = f(n_clusters=Clusters,m=3.8,max_iter=200) 
    Z = np.float32(Z)
    my_model.fit(Z)
    center = my_model.centers
    label = my_model.predict(Z)
    center = np.uint8(center)
    Fres = center[label.flatten()]
    Fres2 = Fres.reshape((image.shape))
    return Fres2
        

def dice_coef(y_true, y_pred):
    y_truef=K.flatten(y_true)
    y_predf=K.flatten(y_pred)
    And=K.sum(y_truef* y_predf)
    return((2* And + 100) / (K.sum(y_truef) + K.sum(y_predf) + 100))

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def iou(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection + 100) / (sum_ - intersection + 100)
    return jac

def waveletDenoise(img):
    noisy = img_as_float(img)
    sigma = 0.12
    sigma_est = estimate_sigma(noisy, channel_axis=-1, average_sigmas=True)
    # Due to clipping in random_noise, the estimate will be a bit smaller than the
    # specified sigma.
    print(f'Estimated Gaussian noise standard deviation = {sigma_est}')

    im_bayes = denoise_wavelet(noisy,wavelet='db2',channel_axis=-1, convert2ycbcr=True,
                               method='BayesShrink', mode='soft',
                               rescale_sigma=True)
    im_visushrink = denoise_wavelet(noisy,wavelet='db2',channel_axis=-1, convert2ycbcr=True,
                                    method='VisuShrink', mode='soft',
                                    sigma=sigma_est, rescale_sigma=True)

    # VisuShrink is designed to eliminate noise with high probability, but this
    # results in a visually over-smooth appearance.  Repeat, specifying a reduction
    # in the threshold by factors of 2 and 4.
    im_visushrink2 = denoise_wavelet(noisy,wavelet='db2' ,channel_axis=-1, convert2ycbcr=True,
                                     method='VisuShrink', mode='soft',
                                     sigma=sigma_est/2, rescale_sigma=True)

    im_visushrink4 = denoise_wavelet(noisy,wavelet='db2',channel_axis=-1, convert2ycbcr=True,
                                     method='VisuShrink', mode='soft',
                                     sigma=sigma_est/4, rescale_sigma=True)
    
   

    Compute PSNR as an indication of image quality
    psnr_noisy = peak_signal_noise_ratio(noisy, noisy)
    psnr_bayes = peak_signal_noise_ratio(noisy, im_bayes)
    psnr_visushrink = peak_signal_noise_ratio(noisy, im_visushrink)
    psnr_visushrink2 = peak_signal_noise_ratio(noisy, im_visushrink2)
    psnr_visushrink4 = peak_signal_noise_ratio(noisy, im_visushrink4)

    ax[0, 0].imshow(noisy)
    ax[0, 0].axis('off')
    ax[0, 0].set_title(f'Noisy\nPSNR={psnr_noisy:0.4g}')
    ax[0, 1].imshow(im_bayes)
    ax[0, 1].axis('off')
    ax[0, 1].set_title(
        f'Wavelet denoising\n(BayesShrink)\nPSNR={psnr_bayes:0.4g}')
    ax[0, 2].imshow(im_visushrink)
    ax[0, 2].axis('off')
    ax[0, 2].set_title(
        'Wavelet denoising\n(VisuShrink, $\\sigma=\\sigma_{est}$)\n'
         'PSNR=%0.4g' % psnr_visushrink)
    ax[1, 0].imshow(noisy)
    ax[1, 0].axis('off')
    ax[1, 0].set_title('Original')
    ax[1, 1].imshow(im_visushrink2)
    ax[1, 1].axis('off')
    ax[1, 1].set_title(
        'Wavelet denoising\n(VisuShrink, $\\sigma=\\sigma_{est}/2$)\n'
         'PSNR=%0.4g' % psnr_visushrink2)
    ax[1, 2].imshow(im_visushrink4)
    ax[1, 2].axis('off')
    ax[1, 2].set_title(
        'Wavelet denoising\n(VisuShrink, $\\sigma=\\sigma_{est}/4$)\n'
         'PSNR=%0.4g' % psnr_visushrink4)
    fig.tight_layout()

    plt.show()

    return [im_bayes,im_visushrink,im_visushrink2,im_visushrink4]



In [2]:
img=cv.imread("/home/boss/Documents/Dataset_BUSI_with_GT/malignant/malignant (3).png",1)
noise_remMedianWeiner(img)

array([[[212, 212, 212],
        [ 52,  52,  52],
        [192, 192, 192],
        ...,
        [143, 143, 143],
        [194, 194, 194],
        [ 48,  48,  48]],

       [[206, 206, 206],
        [ 45,  45,  45],
        [154, 154, 154],
        ...,
        [133, 133, 133],
        [ 97,  97,  97],
        [211, 211, 211]],

       [[201, 201, 201],
        [ 74,  74,  74],
        [ 89,  89,  89],
        ...,
        [ 21,  21,  21],
        [140, 140, 140],
        [162, 162, 162]],

       ...,

       [[202, 202, 202],
        [ 53,  53,  53],
        [207, 207, 207],
        ...,
        [  5,   5,   5],
        [186, 186, 186],
        [142, 142, 142]],

       [[192, 192, 192],
        [169, 169, 169],
        [212, 212, 212],
        ...,
        [ 75,  75,  75],
        [204, 204, 204],
        [115, 115, 115]],

       [[ 50,  50,  50],
        [101, 101, 101],
        [236, 236, 236],
        ...,
        [186, 186, 186],
        [204, 204, 204],
        [179, 179, 179]]